In [1]:
import os
import librosa
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical

# Step 1: Load and preprocess the dataset

# Define the directory where your audio files are located
audio_directory = '/home/jonat/datasets/release_in_the_wild'

# Load the CSV file containing file, speaker, and label information
metadata_file = '/home/jonat/datasets/release_in_the_wild/meta.csv'
metadata_df = pd.read_csv(metadata_file)

# Initialize lists to store MFCCs and corresponding labels
X_mfccs = []
y_labels = []

# Define parameters for MFCC extraction
n_mfcc = 13
max_length = 100  # You can adjust this value as needed

# Loop through each row in the metadata dataframe
for index, row in metadata_df.iterrows():
    file_path = os.path.join(audio_directory, row['file'])
    audio, sr = librosa.load(file_path, sr=None)
    
    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    
    # Ensure that all MFCCs have the same length
    if mfccs.shape[1] < max_length:
        mfccs = np.pad(mfccs, ((0, 0), (0, max_length - mfccs.shape[1])), mode='constant')
    else:
        mfccs = mfccs[:, :max_length]
    
    X_mfccs.append(mfccs)
    y_labels.append(row['label'])

# Convert lists to NumPy arrays
X_mfccs = np.array(X_mfccs)
y_labels = np.array(y_labels)

2023-10-09 07:15:11.665343: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 07:15:11.795195: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-09 07:15:12.287831: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jonat/miniconda3/lib/python3.11/site-packages/nvidia/cudnn/lib:/home/jonat/miniconda3/lib/:/home/jonat/miniconda3/lib/python3.11/site-packages/nvidia/cudnn/lib:/home/jonat/miniconda3/lib/:
2023-10-0

In [2]:
# Convert lists to NumPy arrays
X_mfccs = np.array(X_mfccs)
y_labels = np.array(y_labels)

# Step 2: Encode the labels

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_labels)

# Step 3: Split the dataset into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X_mfccs, y_encoded, test_size=0.2, random_state=42)


In [3]:
# Step 4: Build and train the CNN model

input_shape = (n_mfcc, max_length, 1)  # Input shape for CNN (height, width, channels)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Binary classification, so use sigmoid activation

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# Step 5: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

Epoch 1/10


2023-10-09 07:19:32.658025: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 07:19:33.337186: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22831 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:1a:00.0, compute capability: 7.5
2023-10-09 07:19:33.337951: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22761 MB memory:  -> device: 1, name: NVIDIA TITAN RTX, pci bus id: 0000:68:00.0, compute capability: 7.5
2023-10-09 07:19:34.833072: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600


  1/795 [..............................] - ETA: 22:35 - loss: 20.6434 - accuracy: 0.2812

2023-10-09 07:19:35.348838: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-09 07:19:35.349673: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-09 07:19:35.349688: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-10-09 07:19:35.350455: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-09 07:19:35.350492: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


795/795 [==============================] - 8s 7ms/step - loss: 0.5468 - accuracy: 0.8408 - val_loss: 0.2278 - val_accuracy: 0.9095
Epoch 2/10
795/795 [==============================] - 6s 7ms/step - loss: 0.2281 - accuracy: 0.9116 - val_loss: 0.1921 - val_accuracy: 0.9251
Epoch 3/10
795/795 [==============================] - 6s 7ms/step - loss: 0.1807 - accuracy: 0.9323 - val_loss: 0.1598 - val_accuracy: 0.9386
Epoch 4/10
795/795 [==============================] - 6s 7ms/step - loss: 0.1514 - accuracy: 0.9430 - val_loss: 0.1381 - val_accuracy: 0.9509
Epoch 5/10
795/795 [==============================] - 6s 7ms/step - loss: 0.1315 - accuracy: 0.9507 - val_loss: 0.1222 - val_accuracy: 0.9563
Epoch 6/10
795/795 [==============================] - 6s 7ms/step - loss: 0.1196 - accuracy: 0.9559 - val_loss: 0.1461 - val_accuracy: 0.9449
Epoch 7/10
795/795 [==============================] - 6s 7ms/step - loss: 0.1059 - accuracy: 0.9604 - val_loss: 0.1321 - val_accuracy: 0.9528
Epoch 8/10
795/79

In [8]:
model.save('gautham-model.h5')

: 

In [1]:
import os
import librosa
import numpy as np
from keras.models import load_model

# Define the path to the trained model
model_path = '../jonat/gautham-model.h5'

# Load the trained model
model = load_model(model_path)

# Define the path to the audio file you want to test
audio_file_path_to_test = '/home/jonat/datasets/release_in_the_wild/5.wav'

# Define parameters for MFCC extraction (should match the parameters used during training)
n_mfcc = 13
max_length = 100

# Load and preprocess the sample audio file

# Load the audio file
audio, sr = librosa.load(audio_file_path_to_test, sr=None)

# Extract MFCC features
mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)

# Ensure that the MFCCs have the same length as during training
if mfccs.shape[1] < max_length:
    mfccs = np.pad(mfccs, ((0, 0), (0, max_length - mfccs.shape[1])), mode='constant')
else:
    mfccs = mfccs[:, :max_length]

# Reshape the MFCCs to match the input shape expected by the model
input_shape = (1, n_mfcc, max_length, 1)
mfccs = mfccs.reshape(input_shape)

# Make predictions using the model
predictions = model.predict(mfccs)

# Interpret the predictions
if predictions[0][0] > 0.5:
    result = "Fake"
else:
    result = "Not Fake"

# Print the result
print(f"The audio is classified as: {result}")


2023-10-09 08:16:41.612468: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 08:16:42.432287: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 69 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:1a:00.0, compute capability: 7.5
2023-10-09 08:16:42.432882: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22450 MB memory:  -> device: 1, name: NVIDIA TITAN RTX, pci bus id: 0000:68:00.0, compute capability: 7.5
2023-10-09 08:16:44.507499: E tensorflow/stream_executor/cuda/cuda_dnn.cc:389] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
20

UnimplementedError: Graph execution error:

Detected at node 'sequential/conv2d/Conv2D' defined at (most recent call last):
    File "/home/jonat/miniconda3/envs/ASV/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/jonat/miniconda3/envs/ASV/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/jonat/miniconda3/envs/ASV/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/jonat/miniconda3/envs/ASV/lib/python3.9/site-packages/traitlets/config/application.py", line 1053, in launch_instance
      app.start()
    File "/home/jonat/miniconda3/envs/ASV/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/home/jonat/miniconda3/envs/ASV/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/